    
<p style="text-align:center ; background-color:#F0FFF0;">
     <a href="https://github.com/MoustafaMaher-cyber" target="_blank" >    
     <img src="website.pngىة.png" width="250" alt="Mustafa Maher Logo" align="center" />   
     </a>
</p>
   

### introduction
<p style="font-family:verdana; color:BlueGreen; text-align:center; font-size:120%;">this project is aiming to predict the price of books based on its publisher , auther and previous price.
    Data will be gotten through using web scraping techique from powell <a>https://www.powells.com/</a>
</p>

<p style="font-size:110%;">first we will import libaries  </p>

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from parsel import Selector
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time
import random
chrome_options = Options()
PROXY = "" #HOST:PORT 134.238.252.143:8080
chorme_options = webdriver.ChromeOptions()
chrome_options.add_argument(f'--proxy-server={PROXY}')
service = Service(executable_path="chromdriver")
driver = webdriver.Chrome(service=service, options=chrome_options)

In [ ]:
# here we will get the links of the pages containg data we need , we will get only data about 100 books
baseurl='https://www.powells.com/'
data_list = []
for page in range(1,40,10):
    driver.get(f'https://www.powells.com/used?mpp=50&pg={page}')
    time.sleep(random.uniform(8.5 , 10.9))
    content=driver.page_source 
    soup = BeautifulSoup ( content )
    parts=soup.find('div',class_='ui-sortable')
    links=parts.find_all('a')
    for link in links:
        book_link= baseurl + link['href']
        data_list.append(book_link)

In [ ]:
# in this step will will get data about each book such as title , author etc
all_list =[]
for link in data_list:
    driver.get(link)
    content=driver.page_source 
    soup = BeautifulSoup ( content )
    name=soup.find('h1',class_='book-title').getText()
    author=soup.find('span',itemprop="author").getText()
    publisher=soup.find('dd',class_='publisher').getText()
    try:
        pages=soup.find('dd',class_='pages').text
    except:
        pages = 'None'
    if pages:
        pages.strip()
    price=soup.find('div',class_='price').getText()
    price1=price.split('$')[1]
    try:
        oldprice=soup.find('s').text
    except:
        oldprice='None'
    if oldprice:
        oldprice.strip()
    #old=oldprice.split('$')[1]
    All={
        'Book_Title':name,
        'Author':author,
        'Publisher':publisher,
        'Pages':pages,
        'Price':price1,
        'old_price':oldprice
    }
    all_list.append(All)

### Now we will create a dataframe and do data cleaning

In [ ]:
df=pd.DataFrame(all_list)

In [ ]:
df.to_csv('BooksPre.csv')

In [18]:
import pandas as pd
df=pd.read_csv('BooksPre.csv')
df = df.reset_index(drop=True)
df.drop('Unnamed: 0',axis=1,inplace=True)
df

,Book_Title,Author,Publisher,Pages,Price,old_price
0,They Both Die at the End,Adam Silvera,HARPERCOLLINS PUBLISHERS,416.0,8.95,12.99
1,"Entangled Life: How Fungi Make Our Worlds, Ch...",Merlin Sheldrake,PENGUIN RANDOM HOUSE,368.0,12.95,18.00
2,Astoria: Astor and Jefferson's Lost Pacific E...,Peter Stark,Ecco Press,400.0,11.95,19.99
3,Good Omens: The Nice and Accurate Prophecies ...,Terry Pratchett and Neil Gaiman,HARPERCOLLINS PUBLISHERS,412.0,6.95,9.99
4,It Starts with Us,Colleen Hoover,Atria Books,352.0,12.95,17.99
...,...,...,...,...,...,...
195,Running with Scissors: a Memoir,Augusten Burroughs,MACMILLAN PUBLISHING SERVICES,320.0,8.95,19.00
196,Housekeeping,Marilynne Robinson,ST MARTINS PRESS,224.0,8.95,16.00
197,Gilead,Marilynne Robinson,ST MARTINS PRESS,247.0,8.95,17.00
198,Call Me By Your Name,André Aciman,ST MARTINS PRESS,248.0,9.98,17.00


In [19]:
# check data types 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Book_Title  200 non-null    object 
 1   Author      200 non-null    object 
 2   Publisher   200 non-null    object 
 3   Pages       200 non-null    float64
 4   Price       200 non-null    float64
 5   old_price   200 non-null    float64
dtypes: float64(3), object(3)
memory usage: 9.5+ KB


In [20]:
#let's  check null values
df.isnull().sum()

Book_Title    0
Author        0
Publisher     0
Pages         0
Price         0
old_price     0
dtype: int64

In [21]:
l = []
for i in df['Pages']:
    if i != 'None':
        i=int(i)
    else:
        i=None
    l.append(i)    

In [22]:
df['Pages']=l

In [23]:
df['Price'].unique()

array([ 8.95, 12.95, 11.95,  6.95,  7.95, 17.95, 18.95, 11.96, 25.  ,
       24.  , 10.98, 19.95, 13.95,  9.98, 12.98, 23.  , 11.98, 10.95,
       15.99,  9.95,  5.95,  5.98, 28.  , 15.95, 14.5 ,  5.5 , 16.95,
       18.99, 22.  , 12.5 , 21.  , 14.95,  9.99, 17.  ,  9.5 , 10.36,
        8.98, 10.5 ,  4.95,  8.5 ])

In [24]:
df['Price']=df['Price'].astype('float')    

In [25]:
df.dtypes

Book_Title     object
Author         object
Publisher      object
Pages           int64
Price         float64
old_price     float64
dtype: object

In [26]:
df.head(1)

,Book_Title,Author,Publisher,Pages,Price,old_price
0,They Both Die at the End,Adam Silvera,HARPERCOLLINS PUBLISHERS,416,8.95,12.99


In [27]:
df['old_price'][0]

12.99

In [28]:
df['old_price'].unique()

array([12.99      , 18.        , 19.99      ,  9.99      , 17.99      ,
       16.        , 17.        , 10.99      , 24.99      , 26.99      ,
       19.        , 35.        ,  7.99      , 30.        , 18.95      ,
       26.        , 26.95      , 25.        , 28.99      , 11.99      ,
       18.99      , 14.99      , 16.99      , 17.20176471, 19.95      ,
       32.5       , 15.95      , 20.        , 14.        , 14.95      ,
       25.99      , 39.99      , 27.95      , 21.        , 28.        ,
       29.99      ,  9.95      , 17.95      ,  8.99      , 32.        ,
       16.95      , 31.99      , 21.99      , 13.        , 50.        ])

In [ ]:
v = []
for y in df['old_price']:
    if y != 'None':
        y = y.split('$')[1]
    else:
        y = None
    v.append(y)    

In [ ]:
df['old_price']=v

In [29]:
h=[]
for i in df['old_price']:
    if i != None : 
        i=float(i)
    else:
        i=None
    h.append(i)    

In [30]:
df['old_price']=h

In [31]:
df.head(1)

,Book_Title,Author,Publisher,Pages,Price,old_price
0,They Both Die at the End,Adam Silvera,HARPERCOLLINS PUBLISHERS,416,8.95,12.99


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Book_Title  200 non-null    object 
 1   Author      200 non-null    object 
 2   Publisher   200 non-null    object 
 3   Pages       200 non-null    int64  
 4   Price       200 non-null    float64
 5   old_price   200 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 9.5+ KB


In [33]:
df.isnull().sum()

Book_Title    0
Author        0
Publisher     0
Pages         0
Price         0
old_price     0
dtype: int64

In [34]:
df['Pages'].mode()

0    192
Name: Pages, dtype: int64

In [35]:
import numpy as np
df['Pages'].replace(to_replace=np.nan,value=df['Pages'].mode()[0],inplace=True)

In [36]:
df.isnull().sum()

Book_Title    0
Author        0
Publisher     0
Pages         0
Price         0
old_price     0
dtype: int64

In [37]:
df['old_price'].replace(to_replace=np.nan,value=df['old_price'].mean(),inplace=True)

In [38]:
df.isnull().sum()

Book_Title    0
Author        0
Publisher     0
Pages         0
Price         0
old_price     0
dtype: int64

In [39]:
df.head(5)

,Book_Title,Author,Publisher,Pages,Price,old_price
0,They Both Die at the End,Adam Silvera,HARPERCOLLINS PUBLISHERS,416,8.95,12.99
1,"Entangled Life: How Fungi Make Our Worlds, Ch...",Merlin Sheldrake,PENGUIN RANDOM HOUSE,368,12.95,18.00
2,Astoria: Astor and Jefferson's Lost Pacific E...,Peter Stark,Ecco Press,400,11.95,19.99
3,Good Omens: The Nice and Accurate Prophecies ...,Terry Pratchett and Neil Gaiman,HARPERCOLLINS PUBLISHERS,412,6.95,9.99
4,It Starts with Us,Colleen Hoover,Atria Books,352,12.95,17.99


In [40]:
df.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
195    False
196    False
197    False
198    False
199    False
Length: 200, dtype: bool

In [41]:
df.drop_duplicates(inplace=True)

In [42]:
df.duplicated().any()

False

In [43]:
df.shape

(200, 6)

### splitting data into dependent y and independent x

In [44]:
df.columns

Index(['Book_Title', 'Author', 'Publisher', 'Pages', 'Price', 'old_price'], dtype='object')

In [45]:
x = df.drop('Price',axis=1)
y = df['Price']

In [46]:
x.head()

,Book_Title,Author,Publisher,Pages,old_price
0,They Both Die at the End,Adam Silvera,HARPERCOLLINS PUBLISHERS,416,12.99
1,"Entangled Life: How Fungi Make Our Worlds, Ch...",Merlin Sheldrake,PENGUIN RANDOM HOUSE,368,18.00
2,Astoria: Astor and Jefferson's Lost Pacific E...,Peter Stark,Ecco Press,400,19.99
3,Good Omens: The Nice and Accurate Prophecies ...,Terry Pratchett and Neil Gaiman,HARPERCOLLINS PUBLISHERS,412,9.99
4,It Starts with Us,Colleen Hoover,Atria Books,352,17.99


In [47]:
# we will do label encoding for publisher to get a numircal values
from sklearn.preprocessing import LabelEncoder

In [48]:
le1=LabelEncoder()
x['Book_Title']=le1.fit_transform(x['Book_Title'])

In [49]:
le2=LabelEncoder()
x['Author']=le2.fit_transform(x['Author'])

In [50]:
le3=LabelEncoder()
x['Publisher']=le3.fit_transform(x['Publisher'])

In [51]:
x

,Book_Title,Author,Publisher,Pages,old_price
0,166,0,30,416,12.99
1,52,100,44,368,18.00
2,20,116,22,400,19.99
3,64,136,30,412,9.99
4,81,31,6,352,17.99
...,...,...,...,...,...
195,119,15,39,320,19.00
196,71,94,56,224,16.00
197,59,94,56,247,17.00
198,26,10,56,248,17.00


In [52]:
# we need to make feature scaling 
from sklearn.preprocessing import StandardScaler

In [53]:
#standerscaling
sc=StandardScaler()
x=sc.fit_transform(x)

In [54]:
x=pd.DataFrame(x)

In [55]:
x

,0,1,2,3,4
0,1.237612,-1.822224,-0.745699,0.343082,-0.625143
1,-0.796819,0.511567,-0.055465,0.066658,0.118480
2,-1.367887,0.884974,-1.140118,0.250940,0.413851
3,-0.582668,1.351732,-0.745699,0.320046,-1.070426
4,-0.279288,-1.098749,-1.928957,-0.025483,0.116996
...,...,...,...,...,...
195,0.398856,-1.472156,-0.301977,-0.209765,0.266908
196,-0.457747,0.371540,0.536164,-0.762611,-0.178375
197,-0.671898,0.371540,0.536164,-0.630159,-0.029947
198,-1.260812,-1.588845,0.536164,-0.624400,-0.029947


In [56]:
# now we will split data into training and testing groups
from sklearn.model_selection import train_test_split as tts
x_train,x_test,y_train,y_test=tts(x,y,train_size=0.8,random_state=42)

In [57]:
#feature selection
from mlxtend.feature_selection import SequentialFeatureSelector
best_features={}
def back_sel(model):
    bwfselector = SequentialFeatureSelector(model,
                                            k_features=3,
                                            forward=False,
                                            floating=False,
                                            verbose=2,
                                            scoring='accuracy',
                                            cv=5
                                           ).fit(x_train,y_train)
    model_name=type(model).__name__
    best_features[model_name]={
        'best features index':bwfselector.k_feature_idx_,
        'best feature names':bwfselector.k_feature_names_
    }

In [58]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
from sklearn.svm import SVR
svr = SVR()
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor()
from xgboost import XGBRegressor
xr = XGBRegressor()

In [59]:
back_sel(lr)

C:\Users\Mustafa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Mustafa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Mustafa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "C:\Users\Mustafa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packa

In [60]:
back_sel(svr)

C:\Users\Mustafa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Mustafa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Mustafa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "C:\Users\Mustafa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packa

In [61]:
back_sel(rfr)

C:\Users\Mustafa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Mustafa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Mustafa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "C:\Users\Mustafa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packa

In [62]:
back_sel(gbr)

C:\Users\Mustafa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Mustafa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Mustafa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "C:\Users\Mustafa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packa

In [63]:
back_sel(xr)

C:\Users\Mustafa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Mustafa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Mustafa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "C:\Users\Mustafa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packa

In [64]:
#best features are --->>> 
best_features

{'LinearRegression': {'best features index': (0, 1, 2, 3, 4),
  'best feature names': (0, 1, 2, 3, 4)},
 'SVR': {'best features index': (0, 1, 2, 3, 4),
  'best feature names': (0, 1, 2, 3, 4)},
 'RandomForestRegressor': {'best features index': (0, 1, 2, 3, 4),
  'best feature names': (0, 1, 2, 3, 4)},
 'GradientBoostingRegressor': {'best features index': (0, 1, 2, 3, 4),
  'best feature names': (0, 1, 2, 3, 4)},
 'XGBRegressor': {'best features index': (0, 1, 2, 3, 4),
  'best feature names': (0, 1, 2, 3, 4)}}

so as it appears all 5 features are import but this becouse scaling step although we can choose features that will be curisal for our model 

In [65]:
x=x.iloc[:,1:]

In [66]:
x_train,x_test,y_train,y_test=tts(x,y,train_size=0.8,random_state=42)

#### model selection
<p>Here our target is a continous valued data ,so our problem is regression problem we will use the following models:</p>
<ul>
    <li>LinearRegression</li>
    <li>SVR</li>
    <li>GradientBoostingRegressor</li>
    <li>RandomForestRegressor</li>
    <li>XGBRegressor</li>
</ul>
Already installed 

In [67]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,r2_score
from sklearn.model_selection import cross_val_score

In [68]:
# let's make fuction to train each model and compute its matrics
model_df={}
def model_val(model,x,y):
    X_train,X_test,y_train,y_test=tts(x,y,test_size=0.20,random_state=42)
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    print(f"{model} accuracy is {model.score(X_test,y_test)}")
    score = cross_val_score(model,x,y,cv=5)
    print(f"{model} Avg cross val score is {np.mean(score)}")
    model_name = type(model).__name__#to get the model name only
    model_df[model_name]={'cv':round(np.mean(score)*100,2),
                    'acc':model.score(X_test,y_test)
                    }

In [69]:
model_val(lr,x,y)

LinearRegression() accuracy is 0.7280688377819593
LinearRegression() Avg cross val score is 0.71328301071322


In [70]:
model_df

{'LinearRegression': {'cv': 71.33, 'acc': 0.7280688377819593}}

In [71]:
model_val(svr,x,y)

SVR() accuracy is 0.5174952899636007
SVR() Avg cross val score is 0.5311984184608394


In [72]:
model_df

{'LinearRegression': {'cv': 71.33, 'acc': 0.7280688377819593},
 'SVR': {'cv': 53.12, 'acc': 0.5174952899636007}}

In [73]:
model_val(rfr,x,y)

RandomForestRegressor() accuracy is 0.7015487629537718
RandomForestRegressor() Avg cross val score is 0.6770192256872842


In [74]:
model_df

{'LinearRegression': {'cv': 71.33, 'acc': 0.7280688377819593},
 'SVR': {'cv': 53.12, 'acc': 0.5174952899636007},
 'RandomForestRegressor': {'cv': 67.7, 'acc': 0.7015487629537718}}

In [75]:
model_val(gbr,x,y)

GradientBoostingRegressor() accuracy is 0.7269174104146905
GradientBoostingRegressor() Avg cross val score is 0.6198216939055359


In [76]:
model_df

{'LinearRegression': {'cv': 71.33, 'acc': 0.7280688377819593},
 'SVR': {'cv': 53.12, 'acc': 0.5174952899636007},
 'RandomForestRegressor': {'cv': 67.7, 'acc': 0.7015487629537718},
 'GradientBoostingRegressor': {'cv': 61.98, 'acc': 0.7269174104146905}}

In [77]:
model_val(xr,x,y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...) accuracy is 0.7122633051763116
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_

In [78]:
model_df

{'LinearRegression': {'cv': 71.33, 'acc': 0.7280688377819593},
 'SVR': {'cv': 53.12, 'acc': 0.5174952899636007},
 'RandomForestRegressor': {'cv': 67.7, 'acc': 0.7015487629537718},
 'GradientBoostingRegressor': {'cv': 61.98, 'acc': 0.7269174104146905},
 'XGBRegressor': {'cv': 62.41, 'acc': 0.7122633051763116}}

In [79]:
df_complex = pd.concat({k: pd.DataFrame(v,index=['']) for k, v in model_df.items()}, axis=0)
df_complex.sort_values(ascending=False,by='acc',inplace=True)
df_complex

,,cv,acc
LinearRegression,,71.33,0.728069
GradientBoostingRegressor,,61.98,0.726917
XGBRegressor,,62.41,0.712263
RandomForestRegressor,,67.70,0.701549
SVR,,53.12,0.517495


linear regression is the best acc score model 
##### next we will male hyper paramerters tunning and see if the result will change

In [80]:
from sklearn.model_selection import RandomizedSearchCV
tunned_model_df={}
def model_grid(model,grid):
    rs=RandomizedSearchCV(model,
                  param_distributions=grid,
                   n_iter=20,
                   cv=5,
                   verbose=True
                  )
    rs.fit(x,y)
    print(f'{model} best score is {rs.best_score_}')
    print(f'{model} best parameters are  {rs.best_params_}')
    model_name = type(model).__name__#to get the model name only
    tunned_model_df[model_name]={'best score':round(rs.best_score_*100,2),
                    'best params':rs.best_params_
                    }

In [81]:
lr_grid={'fit_intercept':[True,False],
 'positive':[True,False]
}#done
svr_grid={'C':[.25,.50,.75,1.0],
 'kernel':['linear','poly','rbf', 'sigmoid'],
 'degree':[2,3,4,5]
}#done
rfr_grid={'n_estimators':[10,20,30,40,50,60,70,80,90,100],
 'criterion':['squared_error', 'absolute_error', 'friedman_mse', 'poisson']
}#done
xr_grid={'max_depth': (3, 9),
    'learning_rate': (0.01, 0.5),
    'subsample': (0.1, 1),
    'colsample_bytree': (0.1, 1),
    'lambd': (1, 1)
}#done
gbr_grid={'loss':['squared_error', 'absolute_error', 'huber', 'quantile'],
 'learning_rate':[.1,.3,.5],
 'criterion':['friedman_mse', 'squared_error']
}#done

In [82]:
model_grid(lr,lr_grid)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
LinearRegression() best score is 0.71328301071322
LinearRegression() best parameters are  {'positive': False, 'fit_intercept': True}


C:\Users\Mustafa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 4 is smaller than n_iter=20. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [83]:
tunned_model_df

{'LinearRegression': {'best score': 71.33,
  'best params': {'positive': False, 'fit_intercept': True}}}

In [84]:
model_grid(svr,svr_grid)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
SVR() best score is 0.7127551470954577
SVR() best parameters are  {'kernel': 'sigmoid', 'degree': 5, 'C': 0.5}


In [85]:
tunned_model_df

{'LinearRegression': {'best score': 71.33,
  'best params': {'positive': False, 'fit_intercept': True}},
 'SVR': {'best score': 71.28,
  'best params': {'kernel': 'sigmoid', 'degree': 5, 'C': 0.5}}}

In [86]:
model_grid(rfr,rfr_grid)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
RandomForestRegressor() best score is 0.6902978303693335
RandomForestRegressor() best parameters are  {'n_estimators': 80, 'criterion': 'absolute_error'}


In [87]:
model_grid(xr,xr_grid)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[22:19:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "lambd" } are not used.

[22:19:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "lambd" } are not used.

[22:19:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "lambd" } are not used.

[22:19:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "lambd" } are not used.

[22:19:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Pa

In [88]:
tunned_model_df

{'LinearRegression': {'best score': 71.33,
  'best params': {'positive': False, 'fit_intercept': True}},
 'SVR': {'best score': 71.28,
  'best params': {'kernel': 'sigmoid', 'degree': 5, 'C': 0.5}},
 'RandomForestRegressor': {'best score': 69.03,
  'best params': {'n_estimators': 80, 'criterion': 'absolute_error'}},
 'XGBRegressor': {'best score': 59.11,
  'best params': {'subsample': 1,
   'max_depth': 9,
   'learning_rate': 0.5,
   'lambd': 1,
   'colsample_bytree': 1}}}

In [89]:
model_grid(gbr,gbr_grid)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
GradientBoostingRegressor() best score is 0.6868855526606343
GradientBoostingRegressor() best parameters are  {'loss': 'absolute_error', 'learning_rate': 0.1, 'criterion': 'friedman_mse'}


In [90]:
tunned_model_df

{'LinearRegression': {'best score': 71.33,
  'best params': {'positive': False, 'fit_intercept': True}},
 'SVR': {'best score': 71.28,
  'best params': {'kernel': 'sigmoid', 'degree': 5, 'C': 0.5}},
 'RandomForestRegressor': {'best score': 69.03,
  'best params': {'n_estimators': 80, 'criterion': 'absolute_error'}},
 'XGBRegressor': {'best score': 59.11,
  'best params': {'subsample': 1,
   'max_depth': 9,
   'learning_rate': 0.5,
   'lambd': 1,
   'colsample_bytree': 1}},
 'GradientBoostingRegressor': {'best score': 68.69,
  'best params': {'loss': 'absolute_error',
   'learning_rate': 0.1,
   'criterion': 'friedman_mse'}}}

In [91]:
df_complex2 = pd.concat({k: pd.DataFrame(v,index=['']) for k, v in tunned_model_df.items()}, axis=0)
df_complex2.sort_values(ascending=False,by='best score',inplace=True)
df_complex2

,,best score,best params
LinearRegression,,71.33,NaN
SVR,,71.28,NaN
RandomForestRegressor,,69.03,NaN
GradientBoostingRegressor,,68.69,NaN
XGBRegressor,,59.11,NaN


In [92]:
result=pd.concat([df_complex,df_complex2],axis=1)

In [93]:
result.drop('best params',inplace=True,axis=1)

In [94]:
result

,,cv,acc,best score
LinearRegression,,71.33,0.728069,71.33
GradientBoostingRegressor,,61.98,0.726917,68.69
XGBRegressor,,62.41,0.712263,59.11
RandomForestRegressor,,67.70,0.701549,69.03
SVR,,53.12,0.517495,71.28


In [95]:
result['acc']=result['acc']*100

In [96]:
result

,,cv,acc,best score
LinearRegression,,71.33,72.806884,71.33
GradientBoostingRegressor,,61.98,72.691741,68.69
XGBRegressor,,62.41,71.226331,59.11
RandomForestRegressor,,67.70,70.154876,69.03
SVR,,53.12,51.749529,71.28


In [97]:
result['acc'].max() , result['best score'].max()

(72.80688377819592, 71.33)

#### from the final result the best model is still linearregression with default hyperparameters

In [98]:
lr2 = LinearRegression()
#fit lr with total data
lr2.fit(x,y)

LinearRegression()

In [99]:
df.head(1)

,Book_Title,Author,Publisher,Pages,Price,old_price
0,They Both Die at the End,Adam Silvera,HARPERCOLLINS PUBLISHERS,416,8.95,12.99


In [100]:
x.head(1)

,1,2,3,4
0,-1.822224,-0.745699,0.343082,-0.625143


In [101]:
lr2.predict(x_test)

array([ 6.72434651, 18.85986227, 12.2564067 , 11.15139351,  7.06073333,
        7.07668468, 17.46154074, 11.70513522, 10.89459549, 19.90789286,
       11.40954248, 12.6235061 ,  5.75716293,  7.17427015, 11.09212392,
        7.30513066,  8.8812617 , 21.53069227, 11.77877075, 11.47076985,
        8.31099024, 12.05207909, 18.50222382, 11.36079441,  7.2188495 ,
        8.03349589, 11.03113061, 11.64410456,  7.17624614,  7.11932315,
       15.92267666, 11.88001304, 11.74388791,  5.93740373, 12.54801963,
       11.34634336, 10.03346227,  8.79356369, 15.78970564, 15.79311777])

saving the model

In [102]:
import joblib

In [103]:
joblib.dump(lr2,'book price model')

['book price model']

In [104]:
# creating GUI of our model
from tkinter import *

In [105]:
master = Tk()

In [106]:
def show_entry_fields():
    p1 = le2.transform(e1.get())
    p2 = le3.transform(e2.get())
    p3 = float(e3.get())
    p4 = float(e4.get())
    model = joblib.load('book price model')
    result=model.predict([[p1,p2,p3,p4]])    
    Label(master,text='price is : ').grid(row=6)
    Label(master,text=result).grid(row=7)
    print(np.array(result))


master.title('Book Price Prediction')
label = Label(master , text="Book" ,bg='black',fg='white').grid(row=0,column=2)
Label(master , text="Author" ).grid(row=1)
Label(master , text="Publisher" ).grid(row=2)
Label(master , text="Pages" ).grid(row=3)
Label(master , text="Old Price" ).grid(row=4)
e1=Entry(master)
e2=Entry(master)
e3=Entry(master)
e4=Entry(master)
e1.grid(row=1,column=1)
e2.grid(row=2,column=1)
e3.grid(row=3,column=1)
e4.grid(row=4,column=1)
Button(master,text="Predict",command=show_entry_fields).grid()
mainloop()